# Import Libraries

In [161]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from statistics import mode
from collections import Counter

# Compiled Metrics Results

In [214]:
# calculate RMSE
modes = ['LSTM' ,'GRU','RNN']
EMITEN = 'ANTM'
TARGET_DAYS = [1, 5, 10, 20, 50]
MODEL_LIST = ['Price Prediction', 'Deep Learning', 'Machine Learning']

ensemble_results_dir = f'Ensemble Result/{EMITEN}'
if not os.path.exists(ensemble_results_dir):
    os.makedirs(ensemble_results_dir)

result_file_path = f'{ensemble_results_dir}/{EMITEN}_Results.csv'
header_results = f'emiten,model_type,target_day,accuracy,precision,recall,f1_score'
with open(result_file_path, 'w') as writer:
    writer.write(f'{header_results}\n')

all_predictions = []
for MODEL in MODEL_LIST:
    for TARGET_DAY in TARGET_DAYS:
        if MODEL == 'Price Prediction':
            data_path = f'Result {MODEL}/{EMITEN}/LSTM_{EMITEN}_Target_{TARGET_DAY}.csv'
        else:
            data_path = f'Result {MODEL}/{EMITEN}/{EMITEN}_Target_{TARGET_DAY}.csv'
    
        result_df = pd.read_csv(data_path)
        test_df = result_df[result_df['type'] == 'test']
        testCategoryPredict = test_df['prediction'].values
        testCatY = test_df['ground_truth'].values

        test_acc = round(accuracy_score(list(testCatY), testCategoryPredict), 4)
        test_prec = round(precision_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
        test_rec = round(recall_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
        test_f1 = round(f1_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)

        test_results = f'{EMITEN},{MODEL},{TARGET_DAY},{test_acc},{test_prec},{test_rec},{test_f1}'

        with open(result_file_path, 'a') as writer:
            writer.write('{}\n'.format(test_results))
        all_predictions.append(testCategoryPredict)
        print(f'{MODEL} Target {TARGET_DAY} Done')

Deep Learning Target 1 Done
Deep Learning Target 5 Done
Deep Learning Target 10 Done
Deep Learning Target 20 Done
Deep Learning Target 50 Done
Machine Learning Target 1 Done
Machine Learning Target 5 Done
Machine Learning Target 10 Done
Machine Learning Target 20 Done
Machine Learning Target 50 Done


In [215]:
total_ensemble = len(TARGET_DAYS)
ensemble_preds = np.zeros([all_predictions[0].shape[0], total_ensemble])
ensemble_preds.shape

np_all_prediction = all_predictions[0]
np_all_prediction = np_all_prediction.reshape([-1, 1])

## concat all prediction
for preds in all_predictions[1:]:
    np_all_prediction = np.hstack((np_all_prediction, preds.reshape([-1, 1])))
np_all_prediction = np_all_prediction[:10]

# concat prediction for same targe dat
for i in range(total_ensemble):
    for j in range(np_all_prediction.shape[0]):
        target = 'None'
        tmp = np_all_prediction[j]
        
        # find prediction for the same target day
        tmp = (tmp[i + (total_ensemble * 0)], tmp[i + (total_ensemble * 1)])
        label = Counter(tmp).most_common()[0]
        if label[1] == 1:
            label = 1
        else:
            label = label[0]
        ensemble_preds[j, i] = label

ensemble_preds[:5]

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 2.],
       [1., 2., 1., 2., 2.],
       [1., 1., 1., 1., 1.]])

In [216]:
result_df = pd.read_csv(data_path)
for i in range(total_ensemble):
    result_df = pd.read_csv(data_path)
    test_df = result_df[result_df['type'] == 'test']
    testCatY = test_df['ground_truth'].values
    testCategoryPredict = ensemble_preds[:, i]

    test_acc = round(accuracy_score(list(testCatY), testCategoryPredict), 4)
    test_prec = round(precision_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
    test_rec = round(recall_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
    test_f1 = round(f1_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)

    test_results = f'{EMITEN},Ensemble Model,{TARGET_DAYS[i]},{test_acc},{test_prec},{test_rec},{test_f1}'

    with open(result_file_path, 'a') as writer:
        writer.write('{}\n'.format(test_results))

In [217]:
tmp = pd.read_csv(result_file_path)
tmp

,emiten,model_type,target_day,accuracy,precision,recall,f1_score
0,ANTM,Deep Learning,1,0.5123,0.4629,0.5123,0.4421
1,ANTM,Deep Learning,5,0.5984,0.6173,0.5984,0.6031
2,ANTM,Deep Learning,10,0.7992,0.8011,0.7992,0.7995
3,ANTM,Deep Learning,20,0.8934,0.8944,0.8934,0.8933
4,ANTM,Deep Learning,50,0.9221,0.9263,0.9221,0.9223
5,ANTM,Machine Learning,1,0.5205,0.6274,0.5205,0.4130
6,ANTM,Machine Learning,5,0.5123,0.5762,0.5123,0.4998
7,ANTM,Machine Learning,10,0.7295,0.7530,0.7295,0.7201
8,ANTM,Machine Learning,20,0.7951,0.8003,0.7951,0.7953
9,ANTM,Machine Learning,50,0.8320,0.8394,0.8320,0.8313
